In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv('C:/Users/good/Desktop/HDSC Internship/Regression/energydata_complete.csv')

In [2]:
data.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [3]:
array1=np.array(data['T2'])
array2=np.array(data['T6'])
feature=array1.reshape(-1,1)
target= array2.reshape(-1,1)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(feature, target, test_size=0.3, random_state=1)
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(x_train,y_train)
predicted_values=model.predict(x_test)

In [4]:
from sklearn.metrics import r2_score
r2_score=r2_score(y_test, predicted_values)
r2_score

0.645874003495721

In [5]:
round(r2_score,2)

0.65

In [6]:
del data['date']
del data['lights']

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
normalized_data=pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
normalized_data.columns
feature_df=normalized_data[['T1', 'RH_1', 'T2', 'RH_2', 'T3','RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8','RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed','Visibility', 'Tdewpoint', 'rv1', 'rv2']]
target=normalized_data['Appliances']

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(feature_df, target, test_size=0.3, random_state=42)
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(x_train,y_train)
predicted_values=model.predict(x_test)

In [9]:
#mean absolute error
from sklearn.metrics import mean_absolute_error
mae= mean_absolute_error(y_test,predicted_values)
round(mae, 2)

0.05

In [10]:
#residual sum of squares
import numpy as np
rss=np.sum(np.square(y_test-predicted_values))
round(rss, 2)

45.35

In [11]:
#root mean square error
from sklearn.metrics import mean_squared_error
rmse=np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse, 3)

0.088

In [12]:
#r-square
from sklearn.metrics import r2_score
r2_score=r2_score(y_test, predicted_values)
round(r2_score,2)

0.15

In [13]:
from sklearn.linear_model import Ridge
ridge_reg=Ridge(alpha=0.5)
ridge_reg.fit(x_train,y_train)
from sklearn.linear_model import Lasso
lasso_reg=Lasso(0.001)
lasso_reg.fit(x_train,y_train)

Lasso(alpha=0.001)

In [14]:
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df=pd.DataFrame(weights).reset_index()
    weights_df.columns= ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [15]:
linear_model_weights= get_weights_df(model, x_train, 'Linear_Model_Weights')
ridge_weights=get_weights_df(ridge_reg, x_train, 'Ridge_Weights')
lasso_weights=get_weights_df(lasso_reg, x_train, 'Lasso_Weights')
final_weights=pd.merge(linear_model_weights, ridge_weights, on='Features')
final_weights=pd.merge(final_weights, lasso_weights, on='Features')

In [16]:
final_weights

,Features,Linear_Model_Weights,Ridge_Weights,Lasso_Weights
0,RH_2,-0.456698,-0.401134,-0.000000
1,T_out,-0.321860,-0.250765,0.000000
2,T2,-0.236178,-0.193880,0.000000
3,T9,-0.189941,-0.188584,-0.000000
4,RH_8,-0.157595,-0.156596,-0.000110
5,RH_out,-0.077671,-0.050541,-0.049557
6,RH_7,-0.044614,-0.046291,-0.000000
7,RH_9,-0.039800,-0.041701,-0.000000
8,T5,-0.015657,-0.020727,-0.000000
9,T1,-0.003281,-0.021549,0.000000


In [17]:
ridge_predicted_values=ridge_reg.predict(x_test)
rmse=np.sqrt(mean_squared_error(y_test,ridge_predicted_values))
rmse
round(rmse,3)

0.088

In [18]:
lasso_predicted_values=lasso_reg.predict(x_test)
rmse=np.sqrt(mean_squared_error(y_test,lasso_predicted_values))
round(rmse,3)

0.094